# Part 2.5: 샘플 논문 준비하기

**SNU AI Psychology Workshop - February 2026**

---

## 학습 목표
1. AI 리뷰를 위한 **논문 원고 준비** 방법 이해
2. 본인 논문이 없는 경우 **샘플 논문 생성**
3. 논문을 **Markdown 형식**으로 변환

---

## 논문 준비 옵션

| 옵션 | 설명 | 권장 대상 |
|------|------|----------|
| **A. 본인 논문** | 직접 작성한 초고 사용 | 논문 작성 중인 분 |
| **B. AI 생성** | 관심 주제로 샘플 논문 생성 | 본인 논문이 없는 분 |

---
## Part 1: Setup

In [ ]:
# Cell 1: 환경 설정
import os
import sys

# Colab 환경 감지
try:
    from google.colab import drive
    drive.mount('/content/drive/')
    WORKSHOP_DIR = "/content/drive/MyDrive/aiworkshop_Feb2026/"
    os.makedirs(WORKSHOP_DIR, exist_ok=True)
    os.chdir(WORKSHOP_DIR)
    IN_COLAB = True
    print("🌐 Colab 환경에서 실행 중")
except ImportError:
    current_dir = os.getcwd()
    if current_dir.endswith('notebooks'):
        os.chdir('..')
    WORKSHOP_DIR = os.getcwd()
    IN_COLAB = False
    print("💻 로컬 환경에서 실행 중")

# input 폴더 생성
INPUT_DIR = os.path.join(WORKSHOP_DIR, "input")
os.makedirs(INPUT_DIR, exist_ok=True)

print(f"작업 폴더: {WORKSHOP_DIR}")
print(f"논문 저장 위치: {INPUT_DIR}")

In [ ]:
# Cell 2: API Key 로딩 및 의존성 설치
import os

# Colab에서 필요한 패키지 설치
if IN_COLAB:
    %pip install langchain-google-genai pdfplumber python-docx -q
    print("✅ 패키지 설치 완료")

if IN_COLAB:
    try:
        from google.colab import userdata
        GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    except:
        GEMINI_API_KEY = None
else:
    from dotenv import load_dotenv
    load_dotenv(override=True)
    GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

if GEMINI_API_KEY and not GEMINI_API_KEY.startswith('your_'):
    os.environ['GOOGLE_API_KEY'] = GEMINI_API_KEY
    print("✅ Gemini API Key 로드 완료")
else:
    print("❌ Gemini API Key가 필요합니다")
    print("   발급: https://aistudio.google.com/apikey")

In [ ]:
# Cell 3: 기존 논문 파일 확인
import os

# 지원 형식
SUPPORTED_EXT = ['.md', '.txt', '.pdf', '.docx']

# 기존 파일 확인
existing_files = [f for f in os.listdir(INPUT_DIR) 
                  if os.path.splitext(f)[1].lower() in SUPPORTED_EXT]

print(f"📁 input 폴더 확인: {INPUT_DIR}")
print(f"\n📄 발견된 논문 파일 ({len(existing_files)}개):")

if existing_files:
    for f in existing_files:
        size = os.path.getsize(os.path.join(INPUT_DIR, f)) / 1024
        print(f"   - {f} ({size:.1f} KB)")
    print("\n✅ 기존 파일을 사용하거나, 아래에서 새 논문을 생성할 수 있습니다.")
else:
    print("   (없음)")
    print("\n⚠️ 논문 파일이 없습니다. 아래 옵션 중 하나를 선택하세요.")

---
## 옵션 A: 본인 논문 변환

PDF나 Word 파일을 Markdown으로 변환합니다.

In [ ]:
# Cell 4: 본인 논문 변환 (PDF/DOCX → Markdown)

# ✏️ 변환할 파일 경로 입력
MY_FILE_PATH = ""  # 예: "input/my_paper.pdf" 또는 "input/my_paper.docx"

# ─────────────────────────────────────────

if MY_FILE_PATH and os.path.exists(MY_FILE_PATH):
    ext = os.path.splitext(MY_FILE_PATH)[1].lower()
    text_content = ""
    
    if ext == '.pdf':
        try:
            import pdfplumber
            with pdfplumber.open(MY_FILE_PATH) as pdf:
                for page in pdf.pages:
                    text_content += page.extract_text() or ""
                    text_content += "\n\n"
            print(f"✅ PDF 추출 완료: {len(text_content)} 문자")
        except ImportError:
            print("⚠️ pdfplumber 필요: pip install pdfplumber")
            
    elif ext in ['.docx', '.doc']:
        try:
            import docx
            doc = docx.Document(MY_FILE_PATH)
            text_content = "\n\n".join([para.text for para in doc.paragraphs if para.text.strip()])
            print(f"✅ Word 추출 완료: {len(text_content)} 문자")
        except ImportError:
            print("⚠️ python-docx 필요: pip install python-docx")
            
    elif ext in ['.md', '.txt']:
        with open(MY_FILE_PATH, 'r', encoding='utf-8') as f:
            text_content = f.read()
        print(f"✅ 텍스트 로드 완료: {len(text_content)} 문자")
    
    # my_manuscript.md로 저장
    if text_content:
        output_path = os.path.join(INPUT_DIR, "my_manuscript.md")
        
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(text_content)
        
        print(f"\n💾 저장 완료: {output_path}")
        print(f"\n📄 미리보기 (처음 500자):")
        print("="*50)
        print(text_content[:500])
        print("\n✅ 노트북 3, 4에서 이 논문이 자동으로 사용됩니다!")
else:
    if MY_FILE_PATH:
        print(f"❌ 파일을 찾을 수 없습니다: {MY_FILE_PATH}")
    else:
        print("ℹ️ MY_FILE_PATH에 파일 경로를 입력하세요.")
        print("   예: MY_FILE_PATH = 'input/my_paper.pdf'")

---
## 옵션 B: AI로 샘플 논문 생성

관심 주제를 입력하면 AI가 샘플 논문을 생성합니다.

In [ ]:
# Cell 5: 샘플 논문 생성 설정

# ✏️ 관심 주제 입력 (한글 또는 영어)
RESEARCH_TOPIC = ""  # 예: "소셜미디어와 청소년 우울증의 관계"

# ✏️ 논문 언어 선택
PAPER_LANGUAGE = "한국어"  # "한국어" 또는 "English"

# ✏️ 연구 분야
RESEARCH_FIELD = "심리학"  # 예: "심리학", "인지과학", "신경과학"

# ─────────────────────────────────────────

if RESEARCH_TOPIC:
    print(f"📝 연구 주제: {RESEARCH_TOPIC}")
    print(f"🌐 언어: {PAPER_LANGUAGE}")
    print(f"📚 분야: {RESEARCH_FIELD}")
    print("\n✅ 다음 셀을 실행하여 논문을 생성하세요.")
else:
    print("⚠️ RESEARCH_TOPIC에 관심 주제를 입력하세요.")
    print("\n예시 주제:")
    print("   - 소셜미디어와 청소년 우울증의 관계")
    print("   - 명상이 스트레스 감소에 미치는 영향")
    print("   - AI 챗봇의 심리상담 효과")
    print("   - 수면 부족이 인지 기능에 미치는 영향")
    print("   - 기후 불안과 친환경 행동의 관계")

In [ ]:
# Cell 6: AI 논문 생성
from langchain_google_genai import ChatGoogleGenerativeAI

if not RESEARCH_TOPIC:
    print("❌ 위 셀에서 RESEARCH_TOPIC을 먼저 설정하세요.")
else:
    print(f"🔄 '{RESEARCH_TOPIC}' 주제로 논문 생성 중...")
    print("   (1-2분 소요)")
    
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)
    
    prompt = f"""당신은 {RESEARCH_FIELD} 분야의 연구자입니다.
다음 주제로 학술 논문 초고를 작성하세요.

**주제**: {RESEARCH_TOPIC}
**언어**: {PAPER_LANGUAGE}
**분량**: 약 3000-4000 단어

## 논문 구조

다음 섹션을 포함하세요:

1. **제목** - 구체적이고 학술적인 제목
2. **초록 (Abstract)** - 200-300 단어, 배경/목적/방법/결과/결론
3. **서론 (Introduction)** - 연구 배경, 선행연구, 연구 질문/가설
4. **방법 (Methods)** - 참가자, 측정도구, 절차, 분석방법
5. **결과 (Results)** - 가상의 통계 결과 포함 (예: t=2.45, p<.05)
6. **논의 (Discussion)** - 결과 해석, 한계점, 향후 연구 방향
7. **참고문헌 (References)** - APA 형식으로 5-10개

## 요구사항
- 학술적 톤 유지
- 구체적인 가상 데이터 포함 (N, 평균, 표준편차, 통계치)
- Markdown 형식 사용 (# 제목, ## 섹션, **강조** 등)
- 실제 연구처럼 보이도록 디테일하게 작성

논문을 작성하세요:
"""
    
    response = llm.invoke(prompt)
    generated_paper = response.content
    
    print(f"\n✅ 논문 생성 완료! ({len(generated_paper)} 문자)")
    print("\n" + "="*60)
    print("📄 생성된 논문 미리보기 (처음 1000자):")
    print("="*60)
    print(generated_paper[:1000])
    print("\n...")

In [ ]:
# Cell 7: 생성된 논문 저장
import re

if 'generated_paper' in dir() and generated_paper:
    # my_manuscript.md로 저장 (노트북 3, 4에서 1순위로 사용)
    my_path = os.path.join(INPUT_DIR, "my_manuscript.md")
    
    with open(my_path, 'w', encoding='utf-8') as f:
        f.write(generated_paper)
    
    print(f"💾 저장 완료: {my_path}")
    print(f"   파일 크기: {len(generated_paper)} 문자")
    
    # 주제별 백업 파일도 저장
    safe_topic = re.sub(r'[^\w\s가-힣]', '', RESEARCH_TOPIC)[:30]
    safe_topic = safe_topic.replace(' ', '_')
    backup_path = os.path.join(INPUT_DIR, f"manuscript_{safe_topic}.md")
    
    with open(backup_path, 'w', encoding='utf-8') as f:
        f.write(generated_paper)
    print(f"   + 백업: {backup_path}")
    
    print("\n✅ 노트북 3, 4에서 이 논문이 자동으로 사용됩니다!")
    print("   (my_manuscript.md → 1순위)")
else:
    print("❌ 위 셀을 먼저 실행하세요.")

---
## 정리

In [ ]:
# Cell 8: 최종 확인
print("="*60)
print("📋 논문 준비 상태")
print("="*60)

# 우선순위 확인
my_path = os.path.join(INPUT_DIR, "my_manuscript.md")
sample_path = os.path.join(INPUT_DIR, "sample_manuscript.md")

print("\n📄 논문 로딩 우선순위 (노트북 3, 4):")
if os.path.exists(my_path):
    size = os.path.getsize(my_path) / 1024
    print(f"   1순위: ✅ my_manuscript.md ({size:.1f} KB) ← 사용됨")
else:
    print(f"   1순위: ❌ my_manuscript.md (없음)")

if os.path.exists(sample_path):
    size = os.path.getsize(sample_path) / 1024
    if os.path.exists(my_path):
        print(f"   2순위: ✅ sample_manuscript.md ({size:.1f} KB)")
    else:
        print(f"   2순위: ✅ sample_manuscript.md ({size:.1f} KB) ← 사용됨")
else:
    print(f"   2순위: ❌ sample_manuscript.md (없음)")

# 결과
if os.path.exists(my_path) or os.path.exists(sample_path):
    print("\n🎉 다음 단계: 노트북 3 또는 4를 실행하세요!")
else:
    print("\n❌ 논문 파일이 없습니다.")
    print("   위의 옵션 A 또는 B를 실행하세요.")

---
## 다음 단계

| 노트북 | 내용 | 필요한 파일 |
|--------|------|-------------|
| **3. AI Paper Review Agent** | agentic-paper-review로 리뷰 받기 | `input/my_manuscript.md` |
| **4. Transparent Peer Review** | Few-shot vs Agentic 비교 | `input/my_manuscript.md` + 노트북 3 결과 |

### 본인 논문 직접 지정

노트북 3, 4에서 `MY_PAPER_PATH`를 설정하면 다른 파일도 사용 가능합니다:

```python
MY_PAPER_PATH = "input/다른논문.md"  # 또는 .pdf, .docx
```